# Lab 3A: Deploy Deny Policy for Model Deployments

Apply an Azure Policy to **prevent model deployments** in application team (spoke) resource groups.

## Why This Policy?

| Without Policy | With Policy |
|----------------|-------------|
| Teams can deploy their own models | All models centralized in Landing Zone |
| Fragmented cost tracking | Unified cost management |
| Inconsistent security | Consistent content filters & rate limits |
| No visibility | Full observability via APIM |

## Step 1: Define the Policy

In [1]:
import json

# Policy rule: Deny CognitiveServices model deployments
# Note: The "mode" is specified via --mode flag, not in the rules JSON
POLICY_RULE = {
    "if": {
        "field": "type",
        "equals": "Microsoft.CognitiveServices/accounts/deployments"
    },
    "then": {
        "effect": "deny"
    }
}

print("Policy Rule:")
print(json.dumps(POLICY_RULE, indent=2))

Policy Rule:
{
  "if": {
    "field": "type",
    "equals": "Microsoft.CognitiveServices/accounts/deployments"
  },
  "then": {
    "effect": "deny"
  }
}


## Step 2: Set Variables

In [ ]:
import subprocess

# Get subscription ID
SUBSCRIPTION_ID = subprocess.run(
    'az account show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

# Spoke resource group (from Lab 1B)
SPOKE_RG = "foundry-child-1"

# Policy name
POLICY_NAME = "deny-model-deployments"

print(f"Subscription: {SUBSCRIPTION_ID}")
print(f"Spoke RG:     {SPOKE_RG}")
print(f"Policy Name:  {POLICY_NAME}")

## Step 3: Create Policy Definition at Subscription Level

In [3]:
# Write policy rule to temp file (only the if/then block, not mode)
import tempfile
import os

policy_file = tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False)
json.dump(POLICY_RULE, policy_file)
policy_file.close()

# Create policy definition
!az policy definition create \
    --name "{POLICY_NAME}" \
    --display-name "Deny AI Model Deployments" \
    --description "Prevents deployment of AI models in spoke resource groups. All models must be deployed in the central Landing Zone." \
    --rules "{policy_file.name}" \
    --mode All \
    -o table

os.unlink(policy_file.name)
print("\n✅ Policy definition created")

Description                                                                                                          DisplayName                Mode    Name                    PolicyType
-------------------------------------------------------------------------------------------------------------------  -------------------------  ------  ----------------------  ------------
Prevents deployment of AI models in spoke resource groups. All models must be deployed in the central Landing Zone.  Deny AI Model Deployments  All     deny-model-deployments  Custom

✅ Policy definition created


## Step 4: Assign Policy to Spoke Resource Group

The policy is assigned at the **resource group level** so it only affects spokes, not the Landing Zone.

In [ ]:
# Assign policy to spoke resource group
SCOPE = f"/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{SPOKE_RG}"
POLICY_DEF_ID = f"/subscriptions/{SUBSCRIPTION_ID}/providers/Microsoft.Authorization/policyDefinitions/{POLICY_NAME}"

!az policy assignment create \
    --name "{POLICY_NAME}-{SPOKE_RG}" \
    --display-name "Deny Model Deployments in {SPOKE_RG}" \
    --policy "{POLICY_DEF_ID}" \
    --scope "{SCOPE}" \
    -o table

print(f"\n✅ Policy assigned to {SPOKE_RG}")

## Step 5: Test the Policy

Try to deploy a model in the spoke - it should fail!

In [ ]:
import os

# Load spoke account name from .env
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

SPOKE_ACCOUNT = os.environ.get('SPOKE_ACCOUNT', '')

if SPOKE_ACCOUNT:
    print(f"Attempting to deploy a model to spoke account: {SPOKE_ACCOUNT}")
    print("This should FAIL due to the policy...\n")
    
    # Try to create a deployment (this should fail)
    result = subprocess.run(
        f'''az cognitiveservices account deployment create \
            -g "{SPOKE_RG}" \
            -n "{SPOKE_ACCOUNT}" \
            --deployment-name "test-blocked" \
            --model-name "gpt-4.1-mini" \
            --model-version "2025-04-14" \
            --model-format OpenAI \
            --sku-name GlobalStandard \
            --sku-capacity 1''',
        shell=True, capture_output=True, text=True
    )
    
    if "RequestDisallowedByPolicy" in result.stderr or "denied by policy" in result.stderr.lower():
        print("✅ SUCCESS! Deployment was blocked by policy:")
        print("   'RequestDisallowedByPolicy' - Model deployments are denied in spoke.")
    elif result.returncode != 0:
        print(f"❌ Error (check if policy related): {result.stderr[:500]}")
    else:
        print("⚠️ Deployment succeeded - policy may not be active yet (takes a few minutes)")
else:
    print("⚠️ SPOKE_ACCOUNT not found in .env - run Lab 1B first")

## Step 6: Verify Spoke Can Still USE Models

The policy blocks **deployments**, but using models via APIM connection should still work.

In [6]:
# Test: Can spoke still USE models via Foundry Agent API? (Should work!)
import re
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition

def extract_response(text):
    """Strip DeepSeek <think> tags and leading whitespace"""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return text.strip()

def make_agent_name(team, project, model):
    """Create alphanumeric agent name"""
    return re.sub(r'[^a-zA-Z0-9]', '', f"policy-test-{team}-{project}-{model}")

# Build project endpoint from account and project names
# Format: https://<account>.services.ai.azure.com/api/projects/<project>
SPOKE_ACCOUNT = os.environ.get('SPOKE_ACCOUNT', '')
SPOKE_PROJECT = os.environ.get('SPOKE_PROJECT', '')
APIM_CONNECTION = os.environ.get('APIM_CONNECTION', 'landing-zone-apim')

if SPOKE_ACCOUNT and SPOKE_PROJECT:
    project_endpoint = f"https://{SPOKE_ACCOUNT}.services.ai.azure.com/api/projects/{SPOKE_PROJECT}"
    
    print(f"Testing: Use LZ model via Foundry Agent API")
    print(f"APIM Connection: {APIM_CONNECTION}")
    print()
    
    credential = DefaultAzureCredential()
    client = AIProjectClient(credential=credential, endpoint=project_endpoint)
    openai = client.get_openai_client()
    
    model = "gpt-4.1-mini"
    gateway_model = f"{APIM_CONNECTION}/{model}"
    agent_name = make_agent_name("spoke", "test", model)
    
    try:
        agent = client.agents.create_version(
            agent_name=agent_name,
            definition=PromptAgentDefinition(model=gateway_model, instructions="Be brief.")
        )
        resp = openai.responses.create(
            input="Say 'Policy test passed!' in 5 words or less",
            extra_body={"agent": {"name": agent.name, "version": agent.version, "type": "agent_reference"}}
        )
        text = extract_response(resp.output_text)
        print(f"✅ SUCCESS! Agent responded: {text}")
        print()
        print("Summary:")
        print("   🚫 Model DEPLOYMENT in spoke: BLOCKED by policy")
        print("   ✅ Model USAGE via Agent API: ALLOWED (different resource type)")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("⚠️ SPOKE_ACCOUNT or SPOKE_PROJECT not found in .env")

Testing: Use LZ model via Foundry Agent API
APIM Connection: landing-zone-apim

✅ SUCCESS! Agent responded: Policy test passed!

Summary:
   🚫 Model DEPLOYMENT in spoke: BLOCKED by policy
   ✅ Model USAGE via Agent API: ALLOWED (different resource type)


## Done!

The governance policy is now in place:

| Resource Group | Model Deployments |
|----------------|-------------------|
| Landing Zone (foundry-lz-parent) | ✅ Allowed |
| Spoke (foundry-child-1) | ❌ Blocked by Policy |

### Extending to Other Spokes

To apply this policy to additional spoke resource groups:

```bash
az policy assignment create \
    --name "deny-model-deployments-<spoke-rg>" \
    --policy "<policy-definition-id>" \
    --scope "/subscriptions/<sub-id>/resourceGroups/<spoke-rg>"
```

Or assign at subscription level with exclusions for the Landing Zone.

**Next**: Phase 2 - Build agents with the Agent Service

## Cleanup (Optional)

In [7]:
# Remove policy assignment and definition
# !az policy assignment delete --name "{POLICY_NAME}-{SPOKE_RG}" --scope "{SCOPE}"
# !az policy definition delete --name "{POLICY_NAME}"
# print("✅ Policy removed")